# Machine Learning HW1

## Packages & Data

In [227]:
import time
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from scipy.stats import zscore

%autosave 120

Autosaving every 120 seconds


In [228]:
crx = pd.read_csv('crx.csv')
data = pd.read_csv('data.csv')
crx.label = crx.label.map({'+': 1, '-' : -1})
data.Diagnosis = data.Diagnosis.map({'M' : 1, 'B' : -1})

In [229]:
crx.isna().sum()

att1     0
att2     0
att3     0
att4     0
att5     0
att6     0
att7     0
att8     0
att9     0
att10    0
att11    0
att12    0
att13    0
att14    0
att15    0
label    0
dtype: int64

In [215]:
crx.groupby(['label', 'att13']).size().unstack()

att13,g,p,s
label,,,
-1,338,3,42
1,287,5,15


In [216]:
print(crx.att1.value_counts()) # b/a/?
print(crx.att4.value_counts()) # u/y/?/l
print(crx.att5.value_counts()) # g/p/?/gg
print(crx.att6.value_counts()) # c/q/w/i/aa/ff/k/cc/m/x/d/e/j/?/r
print(crx.att7.value_counts()) # v/h/bb/ff/?/j/z/dd/n/o
print(crx.att9.value_counts()) 
print(crx.att10.value_counts()) # 
print(crx.att12.value_counts()) # f/t
print(crx.att13.value_counts()) # g/s/p

b    468
a    210
?     12
Name: att1, dtype: int64
u    519
y    163
?      6
l      2
Name: att4, dtype: int64
g     519
p     163
?       6
gg      2
Name: att5, dtype: int64
c     137
q      78
w      64
i      59
aa     54
ff     53
k      51
cc     41
m      38
x      38
d      30
e      25
j      10
?       9
r       3
Name: att6, dtype: int64
v     399
h     138
bb     59
ff     57
?       9
j       8
z       8
dd      6
n       4
o       2
Name: att7, dtype: int64
t    361
f    329
Name: att9, dtype: int64
f    395
t    295
Name: att10, dtype: int64
f    374
t    316
Name: att12, dtype: int64
g    625
s     57
p      8
Name: att13, dtype: int64


In [217]:
print(data.Diagnosis.value_counts())

-1    357
 1    212
Name: Diagnosis, dtype: int64


In [218]:
# 資料整理
crx = pd.read_csv('crx.csv')
data = pd.read_csv('data.csv')
crx.label = crx.label.map({'+': 1, '-' : -1})
data.Diagnosis = data.Diagnosis.map({'M' : 1, 'B' : -1})
# for crx
map1 = {'b' : 1, 'a' : -1, '?' : 0}
map4 = {'u' : -1, 'y' : -2, '?' : 0, 'l' : 1}
map5 = {'g' : -1, 'p' : -2, '?' : 0, 'gg' : 2}
map6 = {'c' : 6, 'q' : 10, 'w' : 2, 'i' : 11, 'aa' : 7, 'ff' : 12, 'k' : 8, 
        'cc' : 5, 'm' : 4, 'x' : 8, 'd' : 9, 'e' : 3, '?' : 0, 'r' : 1}
map7 = {'v' : 8, 'h' : 7, 'bb' : 4, 'ff' : 9, '?' : 0, 'j' : 4, 'z' : 5, 'dd' : 3, 'n' : 2, 'o' : 1}
map12 = {'f' : -1, 't' : 1} # same as map9, map10
map13 = {'g' : -2, 's' : -1, 'p' : 1}
crx.att1 = crx.att1.map(map1)
crx.att4 = crx.att4.map(map4)
crx.att5 = crx.att5.map(map5)
crx.att6 = crx.att6.map(map6)
crx.att7 = crx.att7.map(map7)
crx.att9 = crx.att9.map(map12)
crx.att10 = crx.att10.map(map12)
crx.att12 = crx.att12.map(map12)
crx.att13 = crx.att13.map(map13)
crx = crx.replace('?', 0)
crx.att2 = crx.att2.astype(float)
crx.att14 = crx.att14.astype(float)
crx = crx.fillna(0)

In [219]:
crx.head(3)

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,label
0,1,30.83,0.00,-1,-1,2.0,8,1.25,1,1,1,-1,-2,202.0,0,1
1,-1,58.67,4.46,-1,-1,10.0,7,3.04,1,1,6,-1,-2,43.0,560,1
2,-1,24.50,0.50,-1,-1,10.0,7,1.50,1,-1,0,-1,-2,280.0,824,1


In [115]:
data.head(3)

,ID,Diagnosis,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,Unnamed: 31
0,-0.236405,1.297676,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,-0.236403,1.297676,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,0.431741,1.297676,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391


In [220]:
crx_label = crx.label
crx_features = crx.drop(['label'], axis = 1)
crx_features = zscore(crx_features,axis=0)
data_label = data.Diagnosis
data_features = data.drop(['Diagnosis', 'ID'], axis = 1)
data_features = zscore(data_features, axis = 0)

In [158]:
data_features

,3,4,5,6,7,8,9,10,11,12,...,23,24,25,26,27,28,29,30,31,Unnamed: 31
0,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.110995,0.721473,2.060786,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,...,1.901185,0.117700,1.752563,2.015301,0.378365,-0.273318,0.664512,1.629151,-1.360158,-0.709091
565,1.704854,2.085134,1.615931,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,...,1.536720,2.047399,1.421940,1.494959,-0.691230,-0.394820,0.236573,0.733827,-0.531855,-0.973978
566,0.702284,2.045574,0.672676,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,...,0.561361,1.374854,0.579001,0.427906,-0.809587,0.350735,0.326767,0.414069,-1.104549,-0.318409
567,1.838341,2.336457,1.982524,1.735218,1.525767,3.272144,3.296944,2.658866,2.137194,1.043695,...,1.961239,2.237926,2.303601,1.653171,1.430427,3.904848,3.197605,2.289985,1.919083,2.219635


## Basic Linear Classification

In [221]:
def sign(y):
        if y > 0:
            return 1
        else:
            return -1 

In [237]:
class BasicLinearClassification : 
    def __init__(self, iter_n = 1000, lr = 0.001) :
        self.iter_n = iter_n
        self.lr = lr
        self.W = 0
    
    def fit(self, features, label) :
        x = features
        y = label.to_numpy()
        idx = np.random.permutation(x.shape[0])
        x = x.take(idx).to_numpy()
        w = np.zeros(x.shape[1], dtype = float).reshape((-1, 1))
        for k in range(self.iter_n):
            for i in range(x.shape[0]):
                y_hat = sign(np.dot(x[i], w))
                if(y_hat*y[i] <= 0):
                    w += self.lr * (x[i].reshape(-1, 1) * y[i])
        self.W = w
        
    def predict(self, features, label) :
        a = np.dot(features.to_numpy(), self.W)
        a = np.where(a > 0, 1, -1)
        b = np.array(label).reshape(-1, 1)
        return sum(a==b)/label.shape[0]
    def getw(self) :
        return w

In [238]:
LC = BasicLinearClassification()
LC.fit(crx_features, crx_label)
LC.predict(crx_features, crx_label)

array([[ 6.05938910e-15],
       [ 2.65008806e-02],
       [ 1.02421230e-02],
       [-2.19606059e-01],
       [ 1.90532933e-01],
       [ 2.53692437e-02],
       [ 2.39442914e-02],
       [-4.99837404e-02],
       [ 6.00646285e-02],
       [-2.02134077e-02],
       [ 5.14465231e-02],
       [ 2.00710341e-02],
       [ 4.80602903e-02],
       [ 2.36993798e-02],
       [ 5.56169574e-02]])

## Basic Linear Classification With Bias

In [239]:
class BasicLinearClassificationWithBias : 
    def __init__(self, iter_n = 1000, lr = 0.001) :
        self.iter_n = iter_n
        self.lr = lr
        self.W = 0
        self.B = 0
    
    def fit(self, features, label) :
        x = features
        y = label.to_numpy()
        idx = np.random.permutation(x.shape[0])
        x = x.take(idx).to_numpy()
        w = np.zeros(x.shape[1], dtype = float).reshape((-1, 1))
        b = 0
        for k in range(self.iter_n):
            for i in range(x.shape[0]): 
                y_hat = sign(np.dot(x[i], w) + b)
                if(y_hat*y[i] <= 0):
                    w += self.lr * (x[i].reshape(-1, 1) * y[i])
                    b += y[i]
        self.W = w
        self.B = b
        return self.W, self.B
    def predict(self, features, label) :
        
        a = np.dot(features.to_numpy(), self.W) + self.B
        a = np.where(a > 0, 1, -1)
        b = np.array(label).reshape(-1, 1)
        return sum(a==b)/label.shape[0]
    def getwb(self) : 
        return w, b

In [225]:
LCB = BasicLinearClassificationWithBias()
LCB.fit(crx_features, crx_label)
LCB.predict(crx_features, crx_label)

array([0.55507246])

## LSE

In [163]:
# J = WX + B  using LSE
def LSE(features, label) :
    y = label
    x = np.ones((features.shape[0], 1))
    x = np.hstack([x, features])
    w = np.linalg.inv(x.T @ x) @ x.T @ y
    return w

In [164]:
# ckeck accuracy
w =  LSE(crx_features, crx_label)
x = np.ones((crx_features.shape[0], 1))
x = np.hstack([x, crx_features])
a = np.dot(x, w) 
a = np.where(a > 0, 1, -1)
b = np.array(crx_label)
sum(a==b)/crx_label.shape[0]

0.5550724637681159

## Basic Linear Classification With Bias and Voted Perceptron

In [166]:
class VotedPerceptron:
    def __init__(self, n_iter = 100):
        self.n_iter = n_iter
        self.V = []
        self.C = []
        self.k = 0
        self.B = []
    
    def fit(self, x, y):
        x = x.to_numpy()
        y = y.to_numpy()
        k = 0
        v = [np.ones_like(x)[0]]
        c = [0]
        b = [0]
        lr = 0.01
        for epoch in range(self.n_iter): # runs through the data n_iter times
            for i in range(len(x)):
                pred = sign(np.dot(v[k], x[i]) + b[k])
                1 if np.dot(v[k], x[i]) > 0 else -1 # checks the sing of v*k
                if pred == y[i]: # checks if the prediction matches the real Y
                    c[k] += 1 # increments c
                else:
                    v.append(np.add(v[k], y[i] * x[i] * lr))
                    c.append(1)
                    b.append(np.add(b[k], y[i]))
                    k += 1
        self.V = v
        self.C = c
        self.k = k
        self.B = b

    def predict(self, features, label):
        preds = []
        features = features.to_numpy()
        for x in features:
            s = 0
            for w, c, b in zip(self.V, self.C, self.B):
                s = s + c*sign(np.dot(w,x) + b)
            preds.append(sign(s))
        a = np.array(preds)
        b = np.array(label)
        return sum(a==b)/label.shape[0]


In [168]:
vp = VotedPerceptron(n_iter=100)
vp.fit(crx_features, crx_label)
vp.predict(crx_features, crx_label)

0.5550724637681159

In [167]:
vp = VotedPerceptron(n_iter=100)
vp.fit(data_features, data_label)
vp.predict(data_features, data_label)


0.9789103690685413

## Basic Linear Classification With Bias and Large Margin

In [181]:
def BasicLCB(features, label, iter_n = 1000) : 
    x = features
    y = label.to_numpy()
    lr = 0.01
    idx = np.random.permutation(x.shape[0])
    temp = x.take(idx).to_numpy()
    w = np.zeros(x.shape[1], dtype = float).reshape((-1, 1))
    b = np.array((0))
    for k in range(iter_n):
        for i in range(x.shape[0]): 
            y_hat = sign(np.dot(temp[i], w) + b)
            if(y_hat*y[i] <= 0):
                w += lr * (temp[i].reshape(-1, 1) * y[i])
                b += y[i]
    return w, b

In [182]:
# Model 4 : min(w^2)
# want
def LargeMargin(features, label, t = 100) : 
    want = 0
    w_square = np.inf
    for i in range(t) :
        w, b = BasicLCB(features, label)
        want_w = w
        want_b = b
        w_square_t = sum(np.square(w))
        if w_square_t <= w_square :
            want = i
            w_square = w_square_t
            want_w = w
            want_b = b
    return want_w, want_b

In [183]:
# ckeck accuracy
start = time.time()
w, b =  LargeMargin(crx_features, crx_label)
a = np.dot(crx_features.to_numpy(), w) + b
a = np.where(a > 0, 1, -1)
c = np.array(crx_label).reshape(-1, 1)
print(sum(a==c)/crx_label.shape[0])
print(time.time()-start)

[0.55507246]
635.53564453125


In [180]:
a.shape

(690, 15)

## Basic Linear Classification With Bias and Slack Variable

In [189]:
# Model 5 : Slack Variable
# want
def WithSlackVariable(features, label, t = 100, c = 1) : 
    want = 0
    w_square_slack = np.inf
    for i in range(t) :
        w, b = BasicLCB(features, label)
        want_w = w
        want_b = b
        w_square_t = sum(np.square(w))
        y_hat = np.dot(features.to_numpy(), w) + b
        y = np.array(label).reshape(-1, 1)
        t = y*y_hat
        t = 1-t
        d = np.zeros_like(t)
        d = np.hstack([t, d])
        s = sum(d.max(axis = 1))
        if (w_square_t + (c*s)) <= w_square_slack :
            want = i
            w_square_slack = w_square_t + (c*s)
            want_w = w
            want_b = b
    return want_w, want_b

In [190]:
# ckeck accuracy
start = time.time()
w, b =  WithSlackVariable(crx_features, crx_label)
a = np.dot(crx_features.to_numpy(), w) + b
a = np.where(a > 0, 1, -1)
c = np.array(crx_label).reshape(-1, 1)
print(sum(a==c)/crx_label.shape[0])
print(time.time()-start)

[0.55507246]
630.0303938388824


In [204]:
# Linear Classification By Packages
lm = LinearRegression()
lm.fit(crx_features, crx_label)

# 印出係數
print(lm.coef_)

# 印出截距
print(lm.intercept_ )

[-0.00991646  0.00563166 -0.02658135 -0.31067424  0.40283779 -0.06838216
 -0.01867211  0.05265779  0.60500133  0.1370723   0.07086383 -0.02893266
  0.03164677 -0.05538888  0.06576958]
-0.11014492753623172


In [205]:
# ckeck accuracy
a = (np.dot(crx_features.to_numpy(), lm.coef_)) + lm.intercept_
a = np.where(a > 0, 1, -1)
b = np.array(crx_label)
sum(a==b)/crx_label.shape[0]

0.8594202898550725

In [206]:
# SVM
clf=svm.LinearSVC()
clf.fit(crx_features,crx_label)
clf.predict(crx_features)
print(clf.score(crx_features,crx_label))

0.8608695652173913


/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [52]:
X_train, X_test, y_train, y_test = train_test_split(data_features, data_label ,test_size=0.3,random_state = 0)

## For 'CRX' dataset

In [246]:
features = crx_features
label = crx_label
X_train, X_test, y_train, y_test = train_test_split(features, label ,test_size=0.3,random_state = 0)
# Linear lassification
LC = BasicLinearClassification()
LC.fit(X_train, y_train)
print('BasicLinearClassification\'s Accuracy : ', LC.predict(X_test, y_test))
# Linear classification Wuth Bias
LCB = BasicLinearClassificationWithBias()
LCB.fit(X_train, y_train)
print('BasicLinearClassificationWithBias\'s Accuracy : ', LCB.predict(X_test, y_test))
# Least Square Error
w =  LSE(X_train, y_train)
x = np.ones((X_test.shape[0], 1))
x = np.hstack([x, X_test])
a = np.where(np.dot(x, w) > 0, 1, -1)
b = np.array(y_test)
sum(a==b)/y_test.shape[0]
print('Least Square Error\'s Accuracy : ', sum(a==b)/y_test.shape[0])
# min w^2
w_lm, b_lm =  LargeMargin(X_train, y_train)
a = np.where(np.dot(X_test.to_numpy(), w_lm) + b_lm > 0, 1, -1)
c = np.array(y_test).reshape(-1, 1)
print('Minimun w^2 method\'s Accuracy : ', sum(a==c)/y_test.shape[0])
# slack variable
w_sv, b_sv =  WithSlackVariable(X_train, y_train)
a = np.dot(X_test.to_numpy(), w_sv) + b_sv
a = np.where(a > 0, 1, -1)
c = np.array(y_test).reshape(-1, 1)
print('Slack Variable\'s Accuracy : ', sum(a==c)/y_test.shape[0])
# package : linear Classification
lm = LinearRegression()
lm.fit(X_train, y_train)
a = (np.dot(X_test.to_numpy(), lm.coef_)) + lm.intercept_
a = np.where(a > 0, 1, -1)
b = np.array(y_test)
print('Package Linear Classification\'s Accuracy', sum(a==b)/y_test.shape[0])
# package : svm
clf=svm.LinearSVC()
clf.fit(X_train, y_train)
clf.predict(X_test)
print('package : SVM', clf.score(X_test, y_test))

BasicLinearClassification's Accuracy :  [0.4057971]
BasicLinearClassificationWithBias's Accuracy :  [0.52657005]
Least Square Error's Accuracy :  0.8502415458937198
Minimun w^2 method's Accuracy :  [0.32850242]
Slack Variable's Accuracy :  [0.39613527]
Package Linear Classification's Accuracy 0.8502415458937198
package : SVM 0.8502415458937198


/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [247]:
w2, b2 = LCB.getwb()
print(sum(w2 ** 2))
print(sum(w_lm ** 2))

0.9903412220302575
[0.11520616]


## For 'data' dataset

In [242]:
features = data_features
label = data_label
X_train, X_test, y_train, y_test = train_test_split(features, label ,test_size=0.3,random_state = 0)
# Linear lassification
LC = BasicLinearClassification()
LC.fit(X_train, y_train)
print('BasicLinearClassification\'s Accuracy : ', LC.predict(X_test, y_test))
# Linear classification Wuth Bias
LCB = BasicLinearClassificationWithBias()
LCB.fit(X_train, y_train)
print('BasicLinearClassificationWithBias\'s Accuracy : ', LCB.predict(X_test, y_test))
# Least Square Error
w =  LSE(X_train, y_train)
x = np.ones((X_test.shape[0], 1))
x = np.hstack([x, X_test])
a = np.where(np.dot(x, w) > 0, 1, -1)
b = np.array(y_test)
print('Least Square Error\'s Accuracy : ', sum(a==b)/y_test.shape[0])
# min w^2
w_lm, b_lm =  LargeMargin(X_train, y_train)
a = np.where(np.dot(X_test.to_numpy(), w_lm) + b_lm > 0, 1, -1)
c = np.array(y_test).reshape(-1, 1)
print('Minimun w^2 method\'s Accuracy : ', sum(a==c)/y_test.shape[0])
# slack variable
w_sv, b_sv =  WithSlackVariable(X_train, y_train)
a = np.dot(X_test.to_numpy(), w_sv) + b_sv
a = np.where(a > 0, 1, -1)
c = np.array(y_test).reshape(-1, 1)
print('Slack Variable\'s Accuracy : ', sum(a==c)/y_test.shape[0])
# package : linear Classification
lm = LinearRegression()
lm.fit(X_train, y_train)
a = (np.dot(X_test.to_numpy(), lm.coef_)) + lm.intercept_
a = np.where(a > 0, 1, -1)
b = np.array(y_test)
print('Package Linear Classification\'s Accuracy', sum(a==b)/y_test.shape[0])
# package : svm
clf=svm.LinearSVC()
clf.fit(X_train, y_train)
clf.predict(X_test)
print('package : SVM', clf.score(X_test, y_test))

BasicLinearClassification's Accuracy :  [0.78362573]
BasicLinearClassificationWithBias's Accuracy :  [0.63157895]
Least Square Error's Accuracy :  0.9707602339181286
Minimun w^2 method's Accuracy :  [0.66666667]
Slack Variable's Accuracy :  [0.5497076]
Package Linear Classification's Accuracy 0.9707602339181286
package : SVM 0.9415204678362573


### comparison

In [245]:
w2, b2 = LCB.getwb()
print(sum(w2 ** 2))
print(sum(w_lm ** 2))

7.72663779260806
[11.6290047]


## Compare Different C

In [232]:
# Model 5 : Slack Variable
# want
def WithSlackVariable(features, label, t = 100, c = 1) : 
    want = 0
    w_square_slack = np.inf
    for i in range(t) :
        w, b = BasicLCB(features, label)
        want_w = w
        want_b = b
        w_square_t = sum(np.square(w))
        y_hat = np.dot(features.to_numpy(), w) + b
        y = np.array(label).reshape(-1, 1)
        t = y*y_hat
        t = 1-t
        d = np.zeros_like(t)
        d = np.hstack([t, d])
        s = sum(d.max(axis = 1))
        if (w_square_t + (c*s)) <= w_square_slack :
            want = i
            w_square_slack = w_square_t + (c*s)
            want_w = w
            want_b = b
    return want_w, want_b

In [251]:
features = crx_features
label = crx_label
X_train, X_test, y_train, y_test = train_test_split(features, label ,test_size=0.3,random_state = 0)
a = (0.1, 0.3, 0.5, 1, 3, 5, 10)
n = np.zeros((len(a), 2), dtype = float)
count = 0
for i in a : 
    w, b =  WithSlackVariable(X_train, y_train, c = i)
    a = np.dot(X_test.to_numpy(), w) + b
    a = np.where(a > 0, 1, -1)
    c = np.array(y_test).reshape(-1, 1)
    t = sum(a==c)/y_test.shape[0]
    n[count, 0] = i
    n[count, 1] = t
    count += 1
print(n)

[[ 0.1         0.4589372 ]
 [ 0.3         0.57971014]
 [ 0.5         0.54589372]
 [ 1.          0.57004831]
 [ 3.          0.56521739]
 [ 5.          0.55555556]
 [10.          0.66666667]]


In [252]:
features = data_features
label = data_label
X_train, X_test, y_train, y_test = train_test_split(features, label ,test_size=0.3,random_state = 0)
a = (0.1, 0.3, 0.5, 1, 3, 5, 10)
n = np.zeros((len(a), 2), dtype = float)
count = 0
for i in a : 
    w, b =  WithSlackVariable(X_train, y_train, c = i)
    a = np.dot(X_test.to_numpy(), w) + b
    a = np.where(a > 0, 1, -1)
    c = np.array(y_test).reshape(-1, 1)
    t = sum(a==c)/y_test.shape[0]
    n[count, 0] = i
    n[count, 1] = t
    count += 1
print(n)

[[ 0.1         0.62573099]
 [ 0.3         0.62573099]
 [ 0.5         0.6374269 ]
 [ 1.          0.59649123]
 [ 3.          0.61988304]
 [ 5.          0.66081871]
 [10.          0.64327485]]
